### Few shot Learning

소량의 학습만으로 새로운 작업을 학습하거나 일반화하는 것을 의미한다. 모델에 어떻게 반응하기를 원하는지에 대한 예시를 제공하는 것으로, 이 기술은 전체 아키텍처를 건드리지 않고도 모델 맞춤화를 가능하게 한다.

In [32]:
import numpy as np
import pandas as pd
import openai
from openai import OpenAI
import os

with open('../config/api.key') as file :
    lines = file.readlines()
    api_key = lines[0].strip()
    serp_api_key = lines[1].strip()
    langsmith_api_key = lines[2].strip()

openai.api_key = api_key

In [2]:
system_message = """
당신은 AI 마케팅 도우미입니다. 사용자가 새로운 제품 이름에 대한 캐치프레이즈를 만들 수 있도록 도와줍니다.
주어진 제품명에 대해 다음 예시와 비슷한 홍보 문구를 만들어주세요.

Apple - Think Different
Samsung - Do What You Can’t
SK 텔레콤 - 또 다른 세상을 만날 땐 잠시 꺼두셔도 좋습니다.
Amazon - Earth’s Most Customer-Centric Company
Microsoft - Cloud First
Google - Don’t Be Evil

제품명 : 
"""

product_name = 'iPhone'

In [3]:
client = OpenAI(api_key = openai.api_key)

response = client.chat.completions.create(
    model = 'gpt-4o-mini',
    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': product_name},
    ]
)

In [4]:
print(response.choices[0].message.content)

iPhone - Your World in a Touch


대부분의 경우, fewshot learning은 fine tuning을 적절한 도구로 생각할 수 있는 극도로 특수한 시나리오에서도 모델을 맞춤화할 수 있을 만큼 강력하다는 점을 명심하자. 실제로 적절한 few shot learning은 fine tuning process만큼 효과적일 수 있다. 다른 예를 살펴보자. 감정 분석에 특화된 모델을 개발하고자 한다고 가정해 보자. 이를 위해 긍정 또는 부정 등 원하는 결과와 함께 다양한 감정을 가진 일련의 텍스트 예시를 제공한다. 이 예제 세트는 지도 학습 과업을 위한 작은 훈련 세트에 불과하며, fine tuning과 유일한 차이점은 모델의 parameter를 업데이트 하지 않는다는 점이다.

In [16]:
system_message = """
당신은 금융, 경제에 종사하고 있는 사람들의 연설을 전문적으로 분석하는 분석가입니다. 
연설의 감정을 향후 주식시장에 미칠 영향을 기준으로 positive, negative 로 분류합니다.

원문이 한국어가 아닌 경우, 한국어로 번역을 하되 번역된 원문은 제공하지 마세요.

다음 텍스트들을 예시로 사용할 수 있습니다.

텍스트 : "I love this product! It's fantastic and works perfectly."
Positive

텍스트 : "I'm really disappointed with the quality of the food."
Negative

텍스트 : "This is the best day of my life."
Positive

텍스트 : "I can't stand the noise in this restaurant."
Negative

감정만을 구두점 없이 출력합니다.

텍스트 : 
"""

분류기를 테스트하기 위해 Kaggle에서 제공되는 영화 리뷰의 IMDB 데이터베이스를 사용한다.

In [17]:
df = pd.read_csv('../data/IMDB Dataset.csv', encoding = 'utf-8')
df['sentiment'] = df['sentiment'].replace(
    {0: 'negative', 1: 'positive'}
)
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [18]:
df = df.sample(n=10, random_state=42)

In [19]:
df

,review,sentiment
33553,I really liked this Summerslam due to the look...,positive
9427,Not many television shows appeal to quite as m...,positive
199,The film quickly gets to a major chase scene w...,negative
12447,Jane Austen would definitely approve of this o...,positive
39489,Expectations were somewhat high for me when I ...,negative
42724,I've watched this movie on a fairly regular ba...,positive
10822,For once a story of hope highlighted over the ...,positive
49498,"Okay, I didn't get the Purgatory thing the fir...",positive
4144,I was very disappointed with this series. It h...,negative
36958,The first 30 minutes of Tinseltown had my fing...,negative


In [20]:
def process_text(text) : 
    response = client.chat.completions.create(
        model = 'gpt-4o-mini',
        messages = [
            {'role': 'system', 'content': system_message},
            {'role': 'user', 'content': text},
        ]
    )
    return response.choices[0].message.content

df['predicted'] = df['review'].apply(process_text)

In [21]:
df

,review,sentiment,predicted
33553,I really liked this Summerslam due to the look...,positive,Negative
9427,Not many television shows appeal to quite as m...,positive,Positive
199,The film quickly gets to a major chase scene w...,negative,Negative
12447,Jane Austen would definitely approve of this o...,positive,Positive
39489,Expectations were somewhat high for me when I ...,negative,Negative
42724,I've watched this movie on a fairly regular ba...,positive,Positive
10822,For once a story of hope highlighted over the ...,positive,Positive
49498,"Okay, I didn't get the Purgatory thing the fir...",positive,Negative
4144,I was very disappointed with this series. It h...,negative,Negative
36958,The first 30 minutes of Tinseltown had my fing...,negative,Negative


label과 predicted 열을 비교한 결과를 보면 알 수 있듯이, 모델을 fine tuning하지 않고도 모든 리뷰를 정확하게 분류할 수 있다. 이것은 모델 전문화 측면에서 few shot learning 기술을 통해 달성할 수 있는 것의 한 예일 뿐이다.

### Chain of Thought(CoT

사고 연쇄(Chain of Thought)는 중간 추론 단계를 통해 복잡한 추론을 가능하게 하는 기법이다. 또한 모델이 자신의 추론을 설명하도록 유도하여 너무 빠르지 않도록 '강제'함으로써 잘못된 응답을 제공할 위험을 방지한다. LLM이 일반적인 일차 방정식을 풀게 하고 싶다고 가정하자. 이를 위해 LLM이 따라야 할 기본 추론 목록을 제공한다.

In [22]:
system_message = """
일반적인 1차 방정식을 해결하려면 다음 단계를 따르세요.

1. 방정식 식별 : 해결하려는 방정식을 식별합니다.
   방정식은 'ax + b = c' 형태여야 합니다.
   여기서 'a'는 변수의 계수, 'x'는 변수, 'b'는 상수, 'c'는 또 다른 상수입니다.
2. 변수 고립화 : 목표는 변수 'x'를 방정식 한쪽에 고립시키는 것입니다.
   이를 위해 다음 단계를 수행합니다:
   a. 상수 추가 또는 빼기 : 상수를 한쪽으로 이동시키기 위해 양쪽에서 'b'를 더하거나 뺍니다.
   b. 계수로 나누기 : 'x'를 고립시키기 위해 양쪽을 'a'로 나눕니다.
      'a'가 0이면 방정식에는 고유한 해가 없을 수 있습니다.
3. 단순화 : 방정식의 양쪽을 최대한 단순화합니다.
4. 'x'에 대해 해결 : 'x'를 한쪽에 고립시키면 해결책을 얻을 수 있습니다.
   이는 'x = 값' 형태가 될 것입니다. 
5. 해 검토 : 찾은 'x' 값을 원래 방정식에 대입하여 방정식을 만족하는지 확인합니다.
   그렇다면 올바른 해결책을 찾은 것입니다.
6. 해 표현 : 해결책을 명확하고 간결한 형태로 작성합니다.
7. 특수 경우 고려 : 'a'가 0일 때 해가 없거나 무한히 많은 해가 있을 수 있는 특수한 경우를 인지합니다.

방정식 : 
"""

equation = "3x + 5 = 11"

response = client.chat.completions.create(
    model = 'gpt-4o-mini',
    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': equation},
    ]
)

In [23]:
print(response.choices[0].message.content)

주어진 방정식은 \( 3x + 5 = 11 \)입니다. 이 방정식을 해결해 보겠습니다.

1. **방정식 식별**: 방정식은 \( ax + b = c \) 형태이고, 여기서 \( a = 3 \), \( b = 5 \), \( c = 11 \)입니다.

2. **변수 고립화**:
   - **상수 추가 또는 빼기**: 양쪽에서 5를 빼줍니다.
     \[
     3x + 5 - 5 = 11 - 5
     \]
     이는 \( 3x = 6 \)이 됩니다.
     
   - **계수로 나누기**: 양쪽을 3으로 나눕니다.
     \[
     \frac{3x}{3} = \frac{6}{3}
     \]
     그러면 \( x = 2 \)가 됩니다.

3. **단순화**: 현재 방정식은 이미 단순화 되어 있습니다.

4. **'x'에 대해 해결**: \( x = 2 \)입니다.

5. **해 검토**: 원래 방정식에 대입하여 확인합니다.
   \[
   3(2) + 5 = 6 + 5 = 11
   \]
   오른쪽과 왼쪽이 같으므로 올바른 해결책입니다.

6. **해 표현**: 최종 해는 \( x = 2 \)입니다.

7. **특수 경우 고려**: \( a \)는 3이므로 고유한 해가 있습니다.

따라서 주어진 방정식의 해는 \( x = 2 \)입니다.


모델은 meta prompt에 지정된 7단계를 명확하게 따랐으며, 이를 통해 이 과업을 수행하는 데 '시간적 여유'를 가질 수 있다. 응답하기 전에 추론이 필요한 더 복잡한 과업에서 더 나은 결과를 얻기 위해 이 기능을 단답형 prompt와 결합할 수도 있다.

CoT를 사용하면 모델에 중간 추론 단계를 생성하도록 유도한다. 이는 다음 절에서 살펴볼 또 다른 추론 기법의 구성 요소이기도 하다.

### Reason and Act (ReAct)

In [37]:
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent, load_tools
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(
    model_name = 'gpt-4o-mini',
    api_key = openai.api_key,
    
)
tools = load_tools(
    ['serpapi'],
    serpapi_api_key = serp_api_key
)
prompt = hub.pull('hwchase17/react')

agent = create_react_agent(chat, tools, prompt)
agent_executor = AgentExecutor(
    agent = agent,
    tools = tools,
    verbose = True
)

/Users/junghunlee/anaconda3/envs/langchain/lib/python3.11/site-packages/langsmith/client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [38]:
print(prompt.template)

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [39]:
result = agent_executor.invoke(
    {'input': '2024년 8월 5일 미국의 주식시장이 갑자기 하락한 이유는?'}
)
print(result)



> Entering new AgentExecutor chain...
미국의 주식시장이 갑자기 하락한 이유를 찾기 위해 최근 뉴스나 시장 동향을 검색해야 할 것 같습니다. 
Action: Search 
Action Input: "2024년 8월 5일 미국 주식시장 하락 이유" ["미국 경기 둔화가 지표로 확인되며 경기 침체 진단에 증시가 폭락한 가운데 엔화 가치가 급등하며 '엔 캐리 트레이드' 청산 공포까지 커지고 있다. 4일 ...", '미국발 경기침체 우려와 주요국 통화정책 변화, 중동 분쟁과 미국 대통령 선거 등 대외적 불확실성이 해소되지 않은 까닭에 증권시장에선 당분간 변동성이 ...', '증권사들의 프로그램 매매와 미국 경제의 불안이 이유였다고 하지만 정확한 이유는 아닌 것으로 밝혀졌다. 블랙먼데이에 미국 시장 대형주인 IBM, 엑손( ...', '5일 미국발 경기 침체 우려로 국내 증시를 포함한 아시아 증시가 급락한 가운데 미국 나스닥100 선물 지수 역시 급락 중이다. 아시아 증시 폭락이 미국 ...', '달러 인덱스는 안전자산 선호 현상보다 연준의 금리인하 전망에 더욱 초점을 맞추면서 전일비 1.16% 급락한 103.21pt로 거래를 마쳤습니다. WTI 유가의 ...', '미국 경기 침체 우려, 외국인 투자자의 대량 매도, 엔 캐리 트레이드 청산 우려, 인공지능 관련 주식의 수익성 우려, 중동 지정학적 불안 등이 주요 원인 ...', '이번 주(8월 5일~8월 9일) 뉴욕증시는 미국의 경기 침체 우려 속 높은 변동성을 보일 것으로 전망된다.지난주 뉴욕증시는 여러 대형 이벤트 속 롤러 ...', '5일 미국 경기 침체에 대한 우려 등으로 한국 증시가 폭락하며 최악의 하루를 경험했다. 이날 코스피는 전 거래일 대비 8.77% 내린 2,441.55, 코스닥은 ...']Thought: 여러 뉴스 항목에서 2024년 8월 5일 미국 주식시장이 하락한 이유로 미국 경기 둔화, 엔 캐리 트레이드 청산 우려, 외국인 투자자의 대량 매도, 인공지능 관련 주식의